# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Inteligencia Artificial y Aprendizaje Automático**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## **Actividad de las Semanas 5 y 6**
### **Problema de asignación de créditos: South German Dataset.**


## **Nombres y matrículas:**



*   Luis Sebastián Gan Cadena - A01186635

*   Carlos Pano Hernández - A01066264

*   Yocoyani Ehecatzin Pérez Ayala - A01796044



# **Parte I: Partición, análisis y pre-procesamiento de los datos.**

## **Ejercicio 1**

In [14]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
!pip install deep-translator

import pandas as pd
from deep_translator import GoogleTranslator
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, PowerTransformer
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler, SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [15]:
# Si se desean comentar algunos de los Warnings.
import warnings
warnings.filterwarnings('ignore')

In [16]:
# Carga y renombra los nombres de las columnas del alemán al inglés y desplegamos
# de nuevo el DataFrame para ver el resultado obtenido:

# ************* Inlcuye aquí tu código:*****************************
# Leyendo la data
# df = pd.read_csv('data/south+german+credit/SouthGermanCredit.asc', delim_whitespace=True)
df = pd.read_csv('data/south+german+credit/SouthGermanCredit.asc', delim_whitespace=True)

# Traduciendo columnas usando google translator y cambiando los nombres
translator = GoogleTranslator(source='de', target='en')
translated_cols = []
for col in df.columns:
    translated_cols.append(translator.translate(col))

df.columns = translated_cols

# *********** Aquí termina la sección de agregar código *************
df.head(5).T

,0,1,2,3,4
running account,1,1,2,1,1
duration,18,9,12,12,12
moral,4,4,2,4,4
used,2,0,9,0,0
height,1049,2799,841,2122,2171
savings account,1,1,2,1,1
time,2,3,4,3,3
rate,4,2,2,3,4
famges,2,3,2,3,3
guarantor,1,1,1,1,1


## **Ejercicio 2**

In [17]:
# Realiza a continuación una transformación para que la clase negativa (buen cliente)
# quede con el valor de 0 y la clase positiva (mal cliente) quede con el valor de 1.

# ************* Inlcuye aquí tu código:*****************************
target_value = 'credit'

print(f"Revision de la data antes:")
print(df[target_value].value_counts())

dict_to_map = {0:1,
               1:0}

df[target_value] = df[target_value].map(dict_to_map)

# *********** Aquí termina la sección de agregar código *************

print(df[target_value].value_counts())

Revision de la data antes:
credit
1    700
0    300
Name: count, dtype: int64
credit
0    700
1    300
Name: count, dtype: int64


## **Ejercicio 3**

In [18]:
# Realiza una partición solicitada de entrenamiento y prueba.
# Los nombres de los conjuntos deberán ser como se indican en los print de abajo:

# ************* Inlcuye aquí tu código:*****************************

X = df.iloc[:,:-1]
y = df[[target_value]]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=.3, shuffle=True, random_state=1, stratify=y)

# *********** Aquí termina la sección de agregar código *************


# Mostremos las dimensiones de la partición generada:
print("Dimensiones:")
print("Entrenamiento:", Xtrain.shape, ytrain.shape)
print("Prueba:", Xtest.shape, ytest.shape)

# Y el porcentaje de cada clase de la variable de salida:
tmp = ytrain.sum()/ytrain.shape[0]
print("\nPorcentaje clases Positiva: %.2f%%, y Negativa: %.2f%%" % (100*(1-tmp),tmp*100))


Dimensiones:
Entrenamiento: (700, 20) (700, 1)
Prueba: (300, 20) (300, 1)

Porcentaje clases Positiva: 70.00%, y Negativa: 30.00%


### **Con base al porcentaje de los niveles de la variable de salida ¿podemos decir que tenemos un problema de datos desbalanceado? ¿Por qué?**



### ++++++++ Inicia la sección de agregar texto: +++++++++++
Sí, podemos afirmar que hay un problema de datos desbalanceados en este contexto de asignación de créditos. La variable de salida presenta un 70% de casos en los que se otorga el crédito (clase positiva) y un 30% donde no se otorga (clase negativa). Esta disparidad es significativa y puede tener varias implicaciones para el modelo de evaluación de riesgos.

1. **Impacto en la toma de decisiones**: La mayor representación de la clase positiva puede hacer que el modelo desarrolle una tendencia a predecir que los créditos siempre se otorgan. Esto puede resultar en una alta tasa de aciertos, pero también en una alta tasa de falsos positivos, donde se conceden créditos a personas que podrían no ser confiables.

2. **Riesgo financiero**: Si el modelo no identifica adecuadamente a los deudores de alto riesgo (clase negativa), el prestamista podría incurrir en pérdidas significativas debido a impagos. Por otro lado, si se niega el crédito a personas confiables, se pierde la oportunidad de ingresos.

3. **Consecuencias sociales**: La negativa a otorgar créditos puede perpetuar desigualdades, afectando desproporcionadamente a ciertos grupos de personas. Esto puede tener un impacto negativo en la reputación del prestamista y en la confianza del consumidor.

### **Recomendaciones para abordar el desbalance de clases**:

1. **Recolección de datos**: Si es posible, intenta recopilar más datos sobre la clase negativa (no se otorga el crédito) para equilibrar el conjunto de datos.

2. **Sobremuestreo**: Utiliza técnicas como SMOTE (Synthetic Minority Over-sampling Technique) para generar ejemplos sintéticos de la clase negativa, lo que puede ayudar a equilibrar el conjunto de datos.

3. **Submuestreo**: Considera reducir el número de ejemplos de la clase positiva si no es viable aumentar la clase negativa. Esto debe hacerse con cuidado para no perder información valiosa.

4. **Uso de métricas adecuadas**: Evalúa el rendimiento del modelo utilizando métricas como el F1-score, la precisión y el recall, que son más informativas en contextos desbalanceados que la simple precisión.

5. **Modelos ajustados**: Implementa algoritmos que manejen bien el desbalance de clases, como árboles de decisión o modelos de ensemble que permiten ajustar la penalización en las clases.

6. **Análisis de riesgo**: Realiza un análisis de riesgo más profundo que considere no solo la tasa de aprobación, sino también la tasa de impagos y el impacto financiero de cada decisión de crédito.

Implementar estas recomendaciones puede ayudar a mejorar la precisión y la equidad del modelo, minimizando el riesgo tanto de impagos como de rechazos indebidos.
### ++++++++ Termina la sección de agregar texto: +++++++++++



## **Ejercicio 4**

In [19]:
# De acuerdo a la información de la Tabla 3 del artículo de la IEEE
# define las variables correspondientes que se indican a continuación:

# ************* Inlcuye aquí tu código:*****************************

# Variables numéricas:
lista_paper_num = ['duration', 'height', 'age', 'pers']


# Variables ordinales:
lista_paper_ord = ['time', 'rate', 'living time', 'verm', 'bishkred', 'profession']


# Variables nominales:
lista_paper_cat = ['running account', 'moral', 'used', 'savings account', 'famges',
                   'guarantor', 'weitkred', 'residential', 'phone', 'guest died']

# *********** Aquí termina la sección de agregar código *************


## **Ejercicio 5**

### Análisis Variables Numéricas

In [20]:
# Transformaciones que se aplicarán a las variables numéricas usando la clase Pipeline de sklearn:

# ************* Incluye aquí tu código:*****************************

# Variables numéricas:
numericas_pipe = Pipeline(steps=[
    ('minmax_scaler', MinMaxScaler())
])
numericas_pipe_nombres = lista_paper_num

# Variables categóricas nominales:
nominales_pipe = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))  # Codificación OneHot para variables nominales
])
nominales_pipe_nombres = lista_paper_cat

# Variables categóricas ordinales:
ordinales_pipe = Pipeline(steps=[
    ('ordinal', OrdinalEncoder())  # Codificación ordinal para mantener el orden jerárquico
])
ordinales_pipe_nombres = lista_paper_ord

# Conjuntas las transformaciones de todo tipo de variable y
# deja sin procesar aquellas que hayas decidido no transformar:

columnasTransformer = ColumnTransformer(transformers=[
    ('numericas', numericas_pipe, numericas_pipe_nombres),
    ('nominales', nominales_pipe, nominales_pipe_nombres),
    ('ordinales', ordinales_pipe, ordinales_pipe_nombres)],
    remainder='passthrough'  # Dejar sin procesar las columnas restantes
)

# *********** Aquí termina la sección de agregar código *************


## **Ejercicio 6**

In [21]:
# Como se va a utilizar Validación-Cruzada, concatena los conjuntos de entrenamiento y prueba
# en uno nuevo conjunto aumentado que llamaremos trainval para utilizar como entrenamiento:


# ************* Inlcuye aquí tu código:**************************

Xtraintest = pd.concat([Xtrain, Xtest], axis=0)
ytraintest = pd.concat([ytrain, ytest], axis=0)


# *********** Aquí termina la sección de agregar código *************


# Veamos cuántas variables nuevas se introducen con las transformaciones One-Hot-Encoding:
Xtmp = Xtraintest.copy()
tmp = columnasTransformer.fit_transform(Xtmp)
print("Dimensión de las variables de entrada ANTES de las transformaciones:", Xtmp.shape)
print("Dimensión de las variables de entrada DESPUÉS de las transformaciones:", tmp.shape)

Dimensión de las variables de entrada ANTES de las transformaciones: (1000, 20)
Dimensión de las variables de entrada DESPUÉS de las transformaciones: (1000, 41)


## **Ejercicio 7**

In [22]:
# Definimos a continuación la función que llamamos "mis_modelos" que incluye
# todos los modelos que deseamos comparar en el ejercicio.


def mis_modelos():

  modelos, nombres = list(), list()


  # ************* Inlcuye aquí tu código:**************************
  #
  # Deberás incluir en cada modelo los argumentos que consideres
  # adecuados para que cada uno converja y no esté sobreentrenado
  # con respecto a la métrica de la exatitud (accuracy).


  # Regresión Logística - Logistic Regression-LR:
  # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

  modelos.append(LogisticRegression(max_iter=1000, C=1.0, penalty='l1', solver='saga', random_state=42))
  nombres.append('LR')




  # k-Vecinos más Cercanos : k-Nearest-Neighbors-kNN:
  # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

  modelos.append(KNeighborsClassifier(n_neighbors=3, weights='distance', algorithm='auto'))
  nombres.append('kNN')




  # Árbol de decisiones-DecisionTree-DT:
  # https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

  modelos.append(DecisionTreeClassifier(
      max_depth=20,
      min_samples_split=2,
      min_samples_leaf=1,
      criterion='gini',
      max_features=None,
      random_state=42
  ))
  nombres.append('DTree')



  # Bosque Aleatorio-RandomForest-RF:
  # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

  modelos.append(RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, random_state=42))
  nombres.append('RF')




  # XGBoosting:
  # https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier
  # https://xgboost.readthedocs.io/en/stable/parameter.html

  modelos.append(XGBClassifier(n_estimators=200, learning_rate=0.2, max_depth=10, random_state=42))
  nombres.append('XGBoost')




  # Red neuronal de Perceptrón Multicapa-MLP:
  # https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

  modelos.append(MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, alpha=0.01, random_state=42))
  nombres.append('MLP')




  # Máquina de Vectores de Soporte-SVM:
  # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

  modelos.append(SVC(kernel='rbf', C=1.0, gamma=1, random_state=42))
  nombres.append('SVM')

  return modelos, nombres


# Técnica de submuestreo (undersampling) y/o sobremuestreo (versampling) utilizada:
mi_uoSampling = imbPipeline(steps=[
    ('over', RandomOverSampler(random_state=42))
])

Xtv_uo, ytv_uo = mi_uoSampling.fit_resample(Xtraintest, ytraintest)



# *********** Aquí termina la sección de agregar código *******************





# Entrenamos cada uno de los modelos y desplegamos la métricas de Train y Val.

# NOTA: Observa que el método de Validación-Cruzada llama  a los resultados
#       de "validation" como "test":

modelos, nombres = mis_modelos()
resultados = list()

for i in range(len(modelos)):

  # Definimos nuestro pipeline con las transformaciones y los modelos:
  pipeline = Pipeline(steps=[('ct',columnasTransformer),('m',modelos[i])])

  # Aplicaremos validación-cruzada:
  micv = RepeatedStratifiedKFold(n_splits=5,
                                 n_repeats=3,
                                 random_state=5     # agreguemos una semilla para estabilizar resultados.
                                 )


  # Definimos las métricas que desamos recuperar:
  mismetricas = {'miaccuracy':'accuracy','miprecision':'precision','mirecall':'recall',
                 'mifi':'f1','miauc':'roc_auc','migmean':make_scorer(geometric_mean_score)}

  # Llevamos a cabo el entrenamiento:
  scores = cross_validate(pipeline,
                          Xtv_uo,
                          ytv_uo,
                          scoring=mismetricas,
                          cv=micv,
                          return_train_score=True,
                          )

  # Guardemos el resultado de cada modelo para análisis posteriores.
  resultados.append(scores)

  # Desplegamos los valores de las métricas para verificar si no hay
  # subentrenamiento o sobreentrenamiento:
  print('>> %s' % nombres[i])
  for j,k in enumerate(list(scores.keys())):
    if j>1:
      print('\t %s %.3f (%.3f)' % (k, np.mean(scores[k]),np.std(scores[k])))

>> LR
	 test_miaccuracy 0.720 (0.030)
	 train_miaccuracy 0.740 (0.011)
	 test_miprecision 0.722 (0.036)
	 train_miprecision 0.738 (0.012)
	 test_mirecall 0.717 (0.042)
	 train_mirecall 0.744 (0.013)
	 test_mifi 0.719 (0.031)
	 train_mifi 0.741 (0.012)
	 test_miauc 0.792 (0.028)
	 train_miauc 0.823 (0.007)
	 test_migmean 0.719 (0.030)
	 train_migmean 0.740 (0.011)
>> kNN
	 test_miaccuracy 0.775 (0.031)
	 train_miaccuracy 1.000 (0.000)
	 test_miprecision 0.712 (0.030)
	 train_miprecision 1.000 (0.000)
	 test_mirecall 0.930 (0.021)
	 train_mirecall 1.000 (0.000)
	 test_mifi 0.806 (0.024)
	 train_mifi 1.000 (0.000)
	 test_miauc 0.868 (0.024)
	 train_miauc 1.000 (0.000)
	 test_migmean 0.759 (0.035)
	 train_migmean 1.000 (0.000)
>> DTree
	 test_miaccuracy 0.840 (0.023)
	 train_miaccuracy 1.000 (0.001)
	 test_miprecision 0.791 (0.025)
	 train_miprecision 1.000 (0.001)
	 test_mirecall 0.927 (0.024)
	 train_mirecall 1.000 (0.001)
	 test_mifi 0.853 (0.020)
	 train_mifi 1.000 (0.001)
	 test_miauc

In [23]:
# +++++++++ Inicia sección para incluir tu código ++++++++++++++++++++++++


modelo = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, alpha=0.01, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'alpha': [0.0001, 0.001],
    'max_iter': [1000, 2000]
}

# param_grid = {
#     'C': [0.1, 1, 10],
#     'gamma': ['scale', 'auto', 0.1, 1],
#     'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
# }

# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 10]
# }

# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [3, 5, 10],
#     'min_samples_split': [2, 5, 10]
# }

# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (100, 50)],  # Tamaño de las capas ocultas MLP
#     # 'activation': ['tanh', 'relu'],                   # Función de activación
#     # 'solver': ['adam', 'sgd'],                        # Algoritmo de optimización
#     'alpha': [0.0001, 0.001],                         # Regularización L2
#     'learning_rate': ['constant', 'adaptive']         # Tasa de aprendizaje
# }

# param_grid = {
#     'C': [0.01, 0.1, 1, 10, 100],          # Inverso de la regularización
#     'penalty': ['l1', 'l2'],               # Tipo de penalización
#     'solver': ['liblinear', 'saga', 'lbfgs'],        # Algoritmos compatibles con L1 y L2
# }

# param_grid = {
#     'n_neighbors': [3, 5, 7],
#     'weights': ['uniform', 'distance', 'uniform'],
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
# }

# param_grid = {
#     'criterion': ['gini', 'entropy'],        # Función de medida de calidad
#     'max_depth': [3, 5, 10, 20, 30],         # Profundidad máxima del árbol
#     'min_samples_split': [2, 10, 20],        # Muestras mínimas para dividir un nodo
#     'min_samples_leaf': [1, 5, 10],          # Muestras mínimas en una hoja
#     'max_features': [None, 'sqrt', 'log2']
# }


grid = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    n_jobs=-1,
    cv=micv,
    scoring=mismetricas,
    return_train_score=True,
    refit='migmean',
    verbose=1
)


Xct = columnasTransformer.fit_transform(Xtv_uo)

grid_result = grid.fit(Xct, np.ravel(ytv_uo))

mejor_modelo = grid_result.best_estimator_

# +++++++++ Termina sección para incluir tu código ++++++++++++++++++++++++



print("Mejor modelo: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 15 folds for each of 12 candidates, totalling 180 fits


/opt/anaconda3/envs/master-ai-tec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/master-ai-tec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/master-ai-tec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/master-ai-tec/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  war

Mejor modelo: 0.854633 usando {'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'max_iter': 1000}


## **Ejercicio 8**

### **Escribe tus conclusiones finales de la actividad.**

### ++++++++ Inicia la sección de agregar texto: +++++++++++

Para efectos de esta actividad, se compararon varios algoritmos de clasificación en el conjunto de datos: **SouthGermanCredit.asc**, mismo que comprende varios factores de clientes para la aporbación o rechazo de un crédito. Los modelos evaluados incluyen:

- **Regresión Logística (LR)**
- **k-Vecinos más Cercanos (kNN)**
- **Árbol de Decisiones (DTree)**
- **Bosque Aleatorio (RF)**
- **XGBoost**
- **Perceptrón Multicapa (MLP)**
- **Máquina de Vectores de Soporte (SVM)**

Los resultados mostraron que:

- **SVM** logró el mejor rendimiento en términos de **accuracy** (0.909) y **precision** (0.990), aunque presentó un recall más bajo (0.826). Este equilibrio sugiere que el modelo tiene un buen desempeño en la identificación de clases positivas, pero podría mejorar en la captura de todos los casos positivos.

- **XGBoost** también presentó un excelente rendimiento (accuracy de 0.878 y AUC de 0.954), lo que indica que es un modelo robusto para este tipo de tareas.

- Por otro lado, **kNN** y **Árbol de Decisiones** mostraron un notable **overfitting**, con **accuracy** en el conjunto de entrenamiento igual a 1.000, mientras que en el conjunto de prueba fue significativamente menor. Esto resalta la necesidad de un manejo cuidadoso de hiperparámetros para evitar el sobreajuste.

## Manejo de Clases Desbalanceadas

El conjunto de datos original presentaba un desbalance en las clases, lo que podía llevar a que algunos modelos favorecieran la clase mayoritaria. Se utilizó **sobremuestreo** con el método `RandomOverSampler`, lo que permitió equilibrar las clases y mejorar el desempeño de los modelos en métricas como **recall** y **F1-score**.

Este enfoque es crucial en problemas de clasificación donde el desbalance de clases puede distorsionar el rendimiento general de los modelos, haciéndolos menos efectivos en la detección de la clase minoritaria.

## Optimización de Hiperparámetros

Durante el ejercicio, se configuraron los hiperparámetros de cada modelo para promover la convergencia y evitar el sobreajuste:

- Se utilizó un número máximo de iteraciones en la **Regresión Logística** y se ajustaron los parámetros como `max_depth` y `min_samples_split` en los modelos de árbol.
- Para **XGBoost** y **MLP**, se seleccionaron cuidadosamente los parámetros de **learning_rate** y **hidden_layer_sizes** respectivamente.

El ajuste de estos hiperparámetros es esencial para mejorar el rendimiento de los modelos, ya que permite encontrar un balance entre el ajuste del modelo a los datos de entrenamiento y su capacidad para generalizar a datos no vistos.

## Recomendaciones para Mejoras Futuras

- **Exploración de Más Estrategias de Muestreo**: Además de `RandomOverSampler`, sería beneficioso probar otros métodos como `SMOTE` o `ADASYN` para generar muestras sintéticas y mejorar aún más el rendimiento en clases desbalanceadas.

- **Optimización de Hiperparámetros con Búsqueda en Rejilla**: Implementar `GridSearchCV` o `RandomizedSearchCV` para explorar un rango más amplio de hiperparámetros y así maximizar el rendimiento de cada modelo.

- **Ensemble Learning**: Considerar el uso de técnicas de ensamblaje como **Voting Classifier** o **Stacking** para combinar modelos y mejorar la robustez general del sistema.

## Reflexiones Finales

Este ejercicio no solo permitió evaluar el desempeño de distintos algoritmos de clasificación, sino que también enfatizó la importancia de manejar adecuadamente las clases desbalanceadas y optimizar los hiperparámetros para obtener modelos efectivos y confiables. La experiencia adquirida en la selección y evaluación de modelos es invaluable para futuras tareas de modelado en problemas reales.



### ++++++++ Termina la sección de agregar texto. +++++++++++


>> ### **Fin de la Actividad de las Semanas 5 y 6.**